In [ ]:
# default_exp actorcritic.a3c_data

In [ ]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

In [ ]:
#export
import torch.nn.utils as nn_utils
from fastai.torch_basics import *
from fastai.data.all import *
from fastai.basics import *
from dataclasses import field,asdict
from typing import List,Any,Dict,Callable
from collections import deque
import gym
import torch.multiprocessing as mp
from torch.optim import *

from fastrl.data import *
from fastrl.async_data import *
from fastrl.basic_agents import *
from fastrl.learner import *
from fastrl.metrics import *
from fastrl.ptan_extension import *

if IN_NOTEBOOK:
    from IPython import display
    import PIL.Image

In [ ]:
# hide
from nbdev.showdoc import *
from nbdev.imports import *
if not os.environ.get("IN_TEST", None):
    assert IN_NOTEBOOK
    assert not IN_COLAB
    assert IN_IPYTHON

# A3C Datawise

## A3C Model

In [ ]:
# export
class LinearA2C(nn.Module):
    def __init__(self, input_shape, n_actions):
        super(LinearA2C, self).__init__()

        self.policy = nn.Sequential(
            nn.Linear(input_shape[0], 512),
            nn.ReLU(),
            nn.Linear(512, n_actions)
        )

        self.value = nn.Sequential(
            nn.Linear(input_shape[0], 512),
            nn.ReLU(),
            nn.Linear(512, 1)
        )

    def forward(self,x):
        fx=x.float()
        return self.policy(fx),self.value(fx)

## A3C Learner

In [ ]:
# export
def unbatch(batch, net, val_gamma,device='cpu'):
    states = []
    actions = []
    rewards = []
    not_done_idx = []
    last_states = []
    for idx, exp in enumerate(batch):
#         print(exp.state.numpy().shape,int(exp.action),float(exp.reward),exp.last_state.numpy().shape if not bool(exp.done) else None,exp.done)
        states.append(np.array(exp.state.cpu().detach().numpy(), copy=False))
        actions.append(int(exp.action.cpu().detach()))
        rewards.append(float(exp.reward.cpu().detach()))
        if not exp.done:
            not_done_idx.append(idx)
            last_states.append(np.array(exp.last_state.cpu().detach().numpy(), copy=False))
    states_v = torch.FloatTensor(states).to(device)
    actions_t = torch.LongTensor(actions).to(device)
    # handle rewards
    rewards_np = np.array(rewards, dtype=np.float32)
    if not_done_idx:
        last_states_v = torch.FloatTensor(last_states).to(device)
        last_vals_v = net(last_states_v)[1]
        last_vals_np = last_vals_v.data.cpu().numpy()[:, 0]
        rewards_np[not_done_idx] += val_gamma * last_vals_np

    ref_vals_v = torch.FloatTensor(rewards_np).to(device)
    return states_v, actions_t, ref_vals_v

In [ ]:
# export
def loss_func(pred,a,r,sp,d,episode_rewards,learn=None):

    b=list(learn.xb)+list(learn.yb)
    batch=[ExperienceFirstLast(state=b[0][i],action=b[1][i],reward=b[2][i],
                            last_state=b[3][i] if not b[4][i] else None, done=b[4][i],episode_reward=b[5][i])
                            for i in range(len(b[0]))]

    states_v, actions_t, vals_ref_v = \
        unbatch(batch, learn.model,val_gamma=learn.discount**learn.reward_steps, device=default_device())
    batch.clear()

    logits_v, value_v = learn.model(states_v)

    loss_value_v = F.mse_loss(value_v.squeeze(-1), vals_ref_v)

    log_prob_v = F.log_softmax(logits_v, dim=1)
    adv_v = vals_ref_v - value_v.detach()
    log_prob_actions_v = adv_v * log_prob_v[range(128), actions_t]
    loss_policy_v = -log_prob_actions_v.mean()
    prob_v = F.softmax(logits_v, dim=1)
    entropy_loss_v = learn.entropy_beta * (prob_v * log_prob_v).sum(dim=1).mean()
    loss_v = entropy_loss_v + loss_value_v + loss_policy_v

    return loss_v

class A3CLearner(AgentLearner):
    def __init__(self,dls,discount=0.99,entropy_beta=0.01,clip_grad=0.1,reward_steps=1,**kwargs):
        super().__init__(dls,loss_func=partial(loss_func,learn=self),**kwargs)
        self.opt=OptimWrapper(AdamW(self.model.parameters(),eps=1e-3))
        self.model.share_memory()
        self.discount=discount
        self.entropy_beta=entropy_beta
        self.reward_steps=reward_steps
        self.clip_grad=clip_grad

In [ ]:
# export
class A3CTrainer(Callback):
    def after_backward(self):
        nn_utils.clip_grad_norm_(self.learn.model.parameters(),self.learn.clip_grad)

In [ ]:
# export
def data_fit(train_queue=None,items:L=None,agent=None,experience_block=None,
             cancel=None):
    try:
        print('hi1')
        blk=IterableDataBlock(blocks=(experience_block(agent=agent())),
                              splitter=FuncSplitter(lambda x:False))
        dls=blk.dataloaders(items,device=default_device())#,n=128*100)
        while True:
            print('hi')
            for xb in dls[0]:
                xb=[o.cpu().numpy()[0] for o in xb]
                xb=[ExperienceFirstLast(state=xb[0],action=xb[1],reward=xb[2],
                                        last_state=xb[3], # if not xb[4] else None,
                                        done=xb[4],
                                        episode_reward=xb[5])]

                new_rewards = [o.episode_reward for o in xb if o.done and int(o.episode_reward) != 0]
                if new_rewards: train_queue.put(TotalReward(reward=np.mean(new_rewards)))

                for x in xb: train_queue.put(x)
                if cancel.is_set():
                    train_queue.put(None)
                    return None
    finally:
        cancel.set()

In [ ]:
if __name__ == "__main__": 
    import gym
    from fastai.torch_basics import *
    from fastai.data.all import *
    from fastai.basics import *
    from fastrl.data import *
    from fastrl.async_data import *
    from fastrl.basic_agents import *
    from fastrl.learner import *
    from fastrl.metrics import *
    from fastrl.actorcritic.a3c_data import data_fit,A3CTrainer,loss_func,A3CLearner,unbatch,LinearA2C
    
    try:
        mp.set_start_method('spawn')
    except Exception: pass
    
    env = gym.make('CartPole-v1')
    net = LinearA2C(env.observation_space.shape, env.action_space.n).to(default_device())
    net.share_memory()
    
    block=AsyncExperienceBlock(
        agent=partial(ActorCriticAgent, model=net, device='cuda'),
        experience_block=partial(FirstLastExperienceBlock, a=0, seed=0, n_steps=4, dls_kwargs={'bs': 1, 'num_workers': 0,
                                                                      'verbose': False,
                                                                      'indexed': True,
                                                                      'shuffle_train': False,
                                                                      'n':128*1000}),
        data_fit=data_fit,
        n_processes=3,
    )

    blk = IterableDataBlock(blocks=(block), splitter=FuncSplitter(lambda x: False))
    dls = blk.dataloaders(['CartPole-v1'] * 15, bs=128, num_workers=0, verbose=False,
                          indexed=True, shuffle_train=False, n=128*100, device=default_device())

    agent=ActorCriticAgent(model=net)
    learner=A3CLearner(dls,agent=agent,cbs=[A3CTrainer],reward_steps=4,metrics=[AvgEpisodeRewardMetric()])
    learner.fit(10)

starting
starting


epoch,train_loss,train_avg_episode_r,valid_loss,valid_avg_episode_r,time
0,19.194157,23.660611,None,23.660611,00:19
1,26.692215,29.130821,None,29.130821,00:19
2,35.559528,63.534667,None,63.534667,00:19
3,49.418739,79.073167,None,79.073167,00:19
4,55.525867,123.143333,None,123.143333,00:19
5,61.535820,161.861667,None,161.861667,00:19
6,79.512482,186.555000,None,186.555000,00:19
7,84.288086,209.763333,None,209.763333,00:19
8,97.959930,216.406667,None,216.406667,00:19
9,110.241119,207.075000,None,207.075000,00:19


/opt/conda/envs/fastrl/lib/python3.7/site-packages/fastprogress/fastprogress.py:74: UserWarning: Your generator is empty.
  warn("Your generator is empty.")


In [ ]:
from nbdev.export import *
from nbdev.export2html import *
notebook2script()
notebook2html()

Converted 00_core.ipynb.
Converted 01_wrappers.ipynb.
Converted 03_basic_agents.ipynb.
Converted 04_learner.ipynb.
Converted 05a_data.ipynb.
Converted 05b_async_data.ipynb.
Converted 13_metrics.ipynb.
Converted 14_actorcritic.sac.ipynb.
Converted 15_actorcritic.a3c_data.ipynb.
Converted 16_actorcritic.a2c.ipynb.
Converted index.ipynb.
Converted notes.ipynb.
